In [123]:
#Load libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from time import sleep
import time

In [124]:
#Define link and create beautifulSoup object
link = 'https://www.imdb.com/search/title/?title_type=feature&genres=action&genres=Action&explore=genres&ref_=adv_explore_rhs'
response = requests.get(link)
html = response.content
bsObj = BeautifulSoup(html)

In [125]:
#Initialize containers and counters
d, e, j = {}, {}, {}
i = 0

#Test data extraction from one page
for item in bsObj.find_all(attrs={'class': 'lister-item mode-advanced'}):
    j['name'] = item.find(attrs={'class': 'lister-item-header'}).a.text
    j['year'] = item.find(attrs={'class': 'lister-item-header'}). \
    find(attrs={'class': 'lister-item-year text-muted unbold'}).text
    
    if item.find(attrs={'class': 'ratings-bar'}):
        j['rating'] = item.find(attrs={'class': 'ratings-bar'}).find('strong').text
    else:
        j['rating'] = np.nan
    
    if item.find(attrs={'class': 'ratings-bar'}):
        if item.find(attrs={'class': 'ratings-bar'}).find(attrs={'class': 'metascore favorable'}):
            j['metascore'] = item.find(attrs={'class': 'ratings-bar'}).\
            find(attrs={'class': 'metascore favorable'}).text.strip()
        else:
            j['metascore'] = np.nan
    
    if item.find(attrs={'class': 'sort-num_votes-visible'}):
        j['votes'] = item.find(attrs={'class': 'sort-num_votes-visible'}).find(attrs={'name': 'nv'}).string
    else:
        j['votes'] = np.nan
                
    e = {**e, **j}
    d[i] = e
    i+=1

d[1]

{'name': 'Greyhound',
 'year': '(2020)',
 'rating': '7.1',
 'metascore': '64',
 'votes': '41,644'}

In [126]:
#Define function to extract data from page
def getSome(bsObj, start):
    
    d = {}
    if start > 0:
        i = start-1
    else:
        i=start
#     i=start
    e, j = {}, {}
    
    
    for item in bsObj.find_all(attrs={'class': 'lister-item mode-advanced'}):
        try:
            j['name'] = item.find(attrs={'class': 'lister-item-header'}).a.text
            j['year'] = item.find(attrs={'class': 'lister-item-header'}). \
            find(attrs={'class': 'lister-item-year text-muted unbold'}).text

            if item.find(attrs={'class': 'ratings-bar'}):
                j['rating'] = item.find(attrs={'class': 'ratings-bar'}).find('strong').text
            else:
                j['rating'] = np.nan

            if item.find(attrs={'class': 'ratings-bar'}):
                if item.find(attrs={'class': 'ratings-bar'}).find(attrs={'class': 'metascore favorable'}):
                    j['metascore'] = item.find(attrs={'class': 'ratings-bar'}).find(attrs={'class': 'metascore'}).text.strip()
                elif item.find(attrs={'class': 'ratings-bar'}).find(attrs={'class': 'metascore unfavorable'}):
                    j['metascore'] = item.find(attrs={'class': 'ratings-bar'}).find(attrs={'class': 'metascore'}).text.strip()
                elif item.find(attrs={'class': 'ratings-bar'}).find(attrs={'class': 'metascore mixed'}):
                    j['metascore'] = item.find(attrs={'class': 'ratings-bar'}).find(attrs={'class': 'metascore'}).text.strip()
                else:
                    j['metascore'] = np.nan

            if item.find(attrs={'class': 'sort-num_votes-visible'}):
                j['votes'] = item.find(attrs={'class': 'sort-num_votes-visible'}).find(attrs={'name': 'nv'}).string
            else:
                j['votes'] = np.nan

            e = {**e, **j}
            d[i] = e
            i+=1
            j={}
        except:

            continue
        
    return d

In [154]:
#We extract details about each genre on IMDb
mv = []
for tr in bsObj.table.find_all('tr'):
    for td in tr.find_all('td'):
    # print(int(td.contents[-1].strip().replace('(','').replace(')','').replace(',','')))
        mv+=[(td.a.string,
              int(td.contents[-1].strip().replace('(','').replace(')','').replace(',','')))]
mv

[('Action', 46882),
 ('Drama', 14796),
 ('Thriller', 8901),
 ('Adventure', 8314),
 ('Crime', 8248),
 ('Comedy', 6619),
 ('Romance', 4316),
 ('Sci-Fi', 3742),
 ('Fantasy', 2640),
 ('Western', 2587),
 ('Horror', 2388),
 ('War', 1847),
 ('Mystery', 1697),
 ('History', 1074),
 ('Family', 1006),
 ('Animation', 973),
 ('Music', 699),
 ('Sport', 548),
 ('Biography', 410),
 ('Musical', 359),
 ('Film-Noir', 45),
 ('News', 6),
 ('Reality-TV', 4),
 ('Game-Show', 1)]

In [132]:
#We define each genre on IMDb and its number of pages
mv=[('Action', 46882),
 ('Drama', 14796),
 ('Thriller', 8901),
 ('Adventure', 8314),
 ('Crime', 8248),
 ('Comedy', 6619),
 ('Romance', 4316),
 ('Sci-Fi', 3742),
 ('Fantasy', 2640),
 ('Western', 2587),
 ('Horror', 2388),
 ('War', 1847),
 ('Mystery', 1697),
 ('History', 1074),
 ('Family', 1006),
 ('Animation', 973),
 ('Music', 699),
 ('Sport', 548),
 ('Biography', 410),
 ('Musical', 359),
 ('Film-Noir', 45),
 ('News', 6),
 ('Reality-TV', 4),
 ('Game-Show', 1)]

In [131]:
#We extract data with this script from each page, 
#and use selenium to move to the next page
#We save extracted data into a json file for each page
#so that if interrupted, we have what we have done so far.

import json

g = {}
start = 0
for j,k in mv[1:]:

    link2 = "https://www.imdb.com/search/title/?title_type=feature&genres=" + str(j) + "&explore=genres&ref_=adv_prv"
    driver = webdriver.Chrome()
    driver.implicitly_wait(30)
    driver.get(link2)
    soup = BeautifulSoup(link2, 'lxml')
    
    for i in range(int(k/50) + 2):


        start_time = time.time()
        if start>0:
            start=start+1
        g = {**g, **getSome(soup, start)}

        with open('data_imdb_3.json', 'w') as f:
            json.dump(g, f) 
        end_time = time.time()
        print(f"Doing movie number {start} to {start+49} .....in {end_time-start_time}", end='\r')

        start += 50
        try:
            python_button = driver.find_element_by_class_name("next-page")
            python_button.click()
            soup = BeautifulSoup(driver.page_source, 'lxml')
        except:
            break
    

KeyboardInterrupt: 

In [135]:
print(json.dumps(g, sort_keys=True, indent=4))

{
    "50": {
        "metascore": "50",
        "name": "Top Gun",
        "rating": "6.9",
        "votes": "286,035",
        "year": "(1986)"
    },
    "51": {
        "metascore": "75",
        "name": "The Wolf of Wall Street",
        "rating": "8.2",
        "votes": "1,137,077",
        "year": "(2013)"
    },
    "52": {
        "metascore": "47",
        "name": "Midway",
        "rating": "6.7",
        "votes": "55,858",
        "year": "(2019)"
    },
    "53": {
        "metascore": "47",
        "name": "The Batman",
        "rating": NaN,
        "votes": NaN,
        "year": "(2021)"
    },
    "54": {
        "metascore": "69",
        "name": "Inglourious Basterds",
        "rating": "8.3",
        "votes": "1,226,058",
        "year": "(2009)"
    },
    "55": {
        "metascore": "90",
        "name": "Goodfellas",
        "rating": "8.7",
        "votes": "988,301",
        "year": "(1990)"
    },
    "56": {
        "metascore": "94",
        "name": "Pulp Fi

In [138]:
df = pd.DataFrame(g).T

,name,year,rating,metascore,votes
50,Top Gun,(1986),6.9,50,"286,035"
51,The Wolf of Wall Street,(2013),8.2,75,"1,137,077"
52,Midway,(2019),6.7,47,"55,858"
53,The Batman,(2021),NaN,47,NaN
54,Inglourious Basterds,(2009),8.3,69,"1,226,058"


In [ ]:
pd.DataFrame(g).T

In [49]:
dfv = df[~df.duplicated(subset=['name', 'year'], keep='first')]
dfv

,name,year,rating,metascore,votes
0,The Matrix,(1999),8.7,73,"1,624,599"
1,Black Panther,(2018),7.3,88,"589,932"
2,Gemini Man,(2019),5.7,NaN,"72,420"
3,The Last Airbender,(2010),4.1,NaN,"148,633"
4,Tombstone,(1993),7.8,NaN,"122,486"
...,...,...,...,...,...
72933,The Bad News Bears Go to Japan,(1978),3.7,31,"2,634"
72935,Pat and Mike,(1952),7.0,79,"4,974"
72940,Ta Ra Rum Pum,(2007),5.6,49,"4,120"
72942,Balls Out,(2014),4.3,58,"1,362"


In [50]:
dfu = df[~df.duplicated(subset=['name', 'year'], keep='last')]
dfu

,name,year,rating,metascore,votes
0,The Matrix,(1999),8.7,73,"1,624,599"
1,Black Panther,(2018),7.3,88,"589,932"
3,The Last Airbender,(2010),4.1,NaN,"148,633"
6,Deadpool,(2016),8.0,65,"878,685"
7,Thor: Ragnarok,(2017),7.9,74,"560,394"
...,...,...,...,...,...
72942,Balls Out,(2014),4.3,58,"1,362"
72980,The Fencer,(2015),7.1,60,"4,458"
72981,Breakfast with Scot,(2007),6.8,51,"2,897"
72982,The Fish That Saved Pittsburgh,(1979),5.5,40,920


In [52]:
names = set(dfv.name) & set(dfu.name)

In [73]:
dfz = df[df.name.isin(df[df.duplicated(subset=['name', 'year'])].name)].sort_values(by='name')
dfz

,name,year,rating,metascore,votes
11886,#Horror,(2015),3.0,42,"3,334"
21272,#Horror,(2015),3.0,42,"3,334"
36364,#Horror,(2015),3.0,42,"3,334"
17157,'71,(2014),7.2,83,"49,336"
3119,'71,(2014),7.2,83,"49,336"
...,...,...,...,...,...
15647,Æon Flux,(2005),5.5,36,"122,613"
67991,Évolution,(2015),5.9,77,"6,029"
64010,Évolution,(2015),5.9,77,"6,029"
54163,Évolution,(2015),5.9,77,"6,029"


In [78]:
dfz[dfz.metascore.isnull()]

,name,year,rating,metascore,votes
46827,10,(IV) (2011),3.5,NaN,10
1551,10 Minutes Gone,(2019),3.6,NaN,"2,151"
514,"10,000 BC",(2008),5.1,NaN,"124,042"
12453,12 Rounds,(2017),8.0,NaN,16
2489,12 Rounds,(2009),5.6,NaN,"26,891"
...,...,...,...,...,...
1786,iBoy,(2017),6.0,NaN,"19,338"
1147,xXx,(2002),5.9,NaN,"171,275"
627,xXx: Return of Xander Cage,(2017),5.2,NaN,"83,292"
1866,xXx: State of the Union,(2005),4.4,NaN,"66,364"


In [83]:
dfz[~dfz.metascore.isnull()]

,name,year,rating,metascore,votes
11886,#Horror,(2015),3.0,42,"3,334"
21272,#Horror,(2015),3.0,42,"3,334"
36364,#Horror,(2015),3.0,42,"3,334"
17157,'71,(2014),7.2,83,"49,336"
3119,'71,(2014),7.2,83,"49,336"
...,...,...,...,...,...
15647,Æon Flux,(2005),5.5,36,"122,613"
67991,Évolution,(2015),5.9,77,"6,029"
64010,Évolution,(2015),5.9,77,"6,029"
54163,Évolution,(2015),5.9,77,"6,029"


In [89]:
#Number of movies present in both metascore.isnull() and metascore.notnull()
duplicate_movies_with_metascore = \
dfz[~dfz.metascore.isnull()][dfz[~dfz.metascore.isnull()].name.isin((set(dfz[~dfz.metascore.isnull()].name) & set(dfz[dfz.metascore.isnull()].name)))]

In [90]:
duplicate_movies_with_metascore.head()

,name,year,rating,metascore,votes
42449,10,(1979),6.1,68,"14,891"
48836,10,(1979),6.1,68,"14,891"
17113,10 Minutes Gone,(2019),3.6,13,"2,150"
67200,10 Minutes Gone,(2019),3.6,13,"2,150"
33933,10 Minutes Gone,(2019),3.6,13,"2,150"


In [104]:
duplicate_movies_with_metascore[duplicate_movies_with_metascore.duplicated(subset=['name', 'year'])]

,name,year,rating,metascore,votes
48836,10,(1979),6.1,68,"14,891"
67200,10 Minutes Gone,(2019),3.6,13,"2,150"
33933,10 Minutes Gone,(2019),3.6,13,"2,150"
756,"10,000 BC",(2008),5.1,34,"124,044"
24769,"10,000 BC",(2008),5.1,34,"124,044"
...,...,...,...,...,...
17616,xXx: State of the Union,(2005),4.4,37,"66,363"
34233,xXx: State of the Union,(2005),4.4,37,"66,363"
25706,xXx: State of the Union,(2005),4.4,37,"66,363"
24760,Æon Flux,(2005),5.5,36,"122,613"


In [109]:
df_with_meta = duplicate_movies_with_metascore.drop_duplicates(subset=['name'])

In [125]:
df_with_meta.__len__()

1967

In [112]:
dfs = df.drop_duplicates(subset=['name', 'year'])
dfs.head()

,name,year,rating,metascore,votes
0,The Matrix,(1999),8.7,73,"1,624,599"
1,Black Panther,(2018),7.3,88,"589,932"
2,Gemini Man,(2019),5.7,NaN,"72,420"
3,The Last Airbender,(2010),4.1,NaN,"148,633"
4,Tombstone,(1993),7.8,NaN,"122,486"


In [122]:
dfs.__len__()

44239

In [121]:
set(zip(*(dfs.name.values, dfs.year.values))).__len__()

44239

In [123]:
df_final = pd.concat([dfs, df_with_meta])
df_final.head()

,name,year,rating,metascore,votes
0,The Matrix,(1999),8.7,73,"1,624,599"
1,Black Panther,(2018),7.3,88,"589,932"
2,Gemini Man,(2019),5.7,NaN,"72,420"
3,The Last Airbender,(2010),4.1,NaN,"148,633"
4,Tombstone,(1993),7.8,NaN,"122,486"


In [134]:
df_final_2 = df_final[~df_final.duplicated(subset=['name', 'year'], keep='last')]

In [135]:
df_final_2.__len__()

44239

In [141]:
df_final_2[df_final_2.name == 'Tombstone']

,name,year,rating,metascore,votes
72,Tombstone,(1993),7.8,50,"122,486"


In [150]:
df_final_2.reset_index().drop('index', axis=1).to_csv('imdb.csv')

In [151]:
df_final_2.reset_index().drop('index', axis=1).T.to_json('imdb.json')

In [143]:
dfs[dfs.name == 'Tombstone']

,name,year,rating,metascore,votes
4,Tombstone,(1993),7.8,NaN,"122,486"


In [82]:
(set(dfz[~dfz.metascore.isnull()].name) & set(dfz[dfz.metascore.isnull()].name))

{'Skin',
 'Digimon: The Movie',
 'The Avengers',
 'Last Action Hero',
 'Highlander: The Final Dimension',
 'Thieves Like Us',
 'Austin Powers: The Spy Who Shagged Me',
 'The Tuxedo',
 'Flyboys',
 'The Wraith',
 'Eight Legged Freaks',
 'Hotel Artemis',
 'Chennai Express',
 'Every Which Way but Loose',
 'Imprisoned',
 'Breakdown',
 'Wanted',
 'Con Air',
 'Kangaroo Jack',
 'The November Man',
 'Underworld: Evolution',
 'Teenage Mutant Ninja Turtles: Out of the Shadows',
 'Double Take',
 'Macbeth',
 'Escape from New York',
 'Point Blank',
 'Exists',
 'Superman',
 'Knowing',
 'Spartan',
 'The Lego Ninjago Movie',
 "The Hitman's Bodyguard",
 'Blood: The Last Vampire',
 'Virtuosity',
 'Cyrus',
 'Wasteland',
 'RoboCop 3',
 'Hot Pursuit',
 'The Girl on the Train',
 'Shaft',
 'Close',
 'Bloodshot',
 'Rough Cut',
 'Exit Wounds',
 'Stick',
 'Battle Royale',
 'Nowhere to Hide',
 'The Rundown',
 'Faster',
 'The Dead Lands',
 'White House Down',
 'The Burnt Orange Heresy',
 'Year of the Dragon',
 'Fu

In [72]:
df.__len__(), dfv.__len__()

(63187, 44239)